In [ ]:
import os
os.chdir("/content/drive/MyDrive/MADE/work/hakaton")


In [ ]:
!unzip 1-20220413T070901Z-001.zip -d /content/data/
!unzip 1-20220413T070901Z-002.zip -d /content/data/
!unzip 1-20220413T070901Z-003.zip -d /content/data/
!unzip 3-20220413T073257Z-001.zip -d /content/data/
!unzip drova-20220413T073301Z-001.zip -d /content/data/

In [ ]:
!unzip test-20220413T104559Z-001.zip -d /content/data/test/
!unzip test-20220413T104559Z-002.zip -d /content/data/test/


Производились следующие эксперименты, использовались сетки resnext50 (слишком тяжелая для данной задачи), efficient_net (очень медленно обучалась), resnet34, resnet50 - не показали себя лучше,чем resnet18. Менялись аугентации на более или менее вероятные, менялся размер CROP_SIZE (больше и меньше 224), менялось расписание шага, пробовала обычную crossentropy loss (взвешенную и невзвешенную) и focal_loss, меняла количество фолдов (5,4,3), коэффициент l2 регуляризации (0.01, 0.001, 0.0001)

**Описание параметров лучшей модели:**  
Resnet18, pretrained, все веса разморожены.  
```
*   Loss Function : FocalLoss 
*   Validation: StratifiedKFold с количеством фолдов 4 
*   CROP_SIZE = 190 (вместо 224)
*   lr = 3e-4 до 1e-5
*   batch_size = 64
*   Оптимизатор AdamW, l2 reg = 0.0001
```






Трансформации:
```   
augmenter = albumentations.Compose([
    albumentations.ShiftScaleRotate(rotate_limit=25, p=0.7),
    albumentations.RandomBrightnessContrast(p=0.4),
    albumentations.RandomGamma(p=0.4),
    albumentations.Blur(blur_limit=2, p=0.1),
    albumentations.GaussNoise((10, 100), p=0.2),
    albumentations.HorizontalFlip(p=0.5)
])
```


# Создание датасетов, dataloaders, функций для обучения и валидации

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import random
import math
import time
import torch.nn.functional as F
import glob

from tqdm.notebook import tqdm
from torch import optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import numpy as np
from  scipy import stats
import scipy
import cv2

import torch.optim as optim
import torchvision.models as models
from tqdm.notebook import tqdm
from torch.nn import functional as fnn





import pandas as pd
import PIL
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
def seed_worker(worker_id):
    worker_seed = 1#torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

np.random.seed(1)
torch.manual_seed(1)
random.seed(1)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
#torch.use_deterministic_algorithms(True)
g = torch.Generator()
g.manual_seed(0)

In [ ]:
class Args:
  def __init__(self): #(data_path, epoch, batch_siz, image_size, learning_rate, weight_deca, learning_rate, learning_rate_gamma, weight_bce, load, output_dir)
    self.data_path = "/content/drive/MyDrive/MADE/semester2/CV/contest02/data/"
    self.epochs = 2
    self.batch_size = 64
    self.lr= 3e-4
    self.weight_decay= 1e-6
    self.learning_rate=1e-3
    self.learning_rate_gamma=None
    self.weight_bce=1
    self.load=None
    self.gpu = True
    self.output_dir="/content/drive/MyDrive/MADE/Project/RACNN_models/"
    self.data_dir ="./data_preprocessed_python/"# "/content/drive/MyDrive/MADE/Project/train/physionet.org/"
args = Args()   

In [ ]:
NUM_CLASSES = 3
CROP_SIZE = 190
device = torch.device("cuda:0") if args.gpu and torch.cuda.is_available() else torch.device("cpu")

In [ ]:
files_glob = []
root = '/content/data/'

In [ ]:
files = glob.glob(os.path.join(root, f"*/*.png"))

In [ ]:
print(files)

['/content/data/drova/IMG_1071.png', '/content/data/drova/IMG_1104.png', '/content/data/drova/IMG_1124.png', '/content/data/drova/IMG_1114.png', '/content/data/drova/IMG_1115.png', '/content/data/drova/IMG_1053.png', '/content/data/drova/IMG_1051.png', '/content/data/drova/IMG_1057.png', '/content/data/drova/IMG_1084.png', '/content/data/drova/IMG_1061.png', '/content/data/drova/IMG_1068.png', '/content/data/drova/IMG_1083.png', '/content/data/drova/IMG_1120.png', '/content/data/drova/IMG_1112.png', '/content/data/drova/IMG_1060.png', '/content/data/drova/IMG_1056.png', '/content/data/drova/IMG_1046.png', '/content/data/drova/IMG_1105.png', '/content/data/drova/IMG_1072.png', '/content/data/drova/IMG_1069.png', '/content/data/drova/IMG_1042.png', '/content/data/drova/IMG_1099.png', '/content/data/drova/IMG_1041.png', '/content/data/drova/IMG_1055.png', '/content/data/drova/IMG_1122.png', '/content/data/drova/IMG_1076.png', '/content/data/drova/IMG_1123.png', '/content/data/drova/IMG_10

In [ ]:
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import f1_score, accuracy_score

labels = []
ch = ['1', '3', 'd']
for file in files:
  folder = file[14]
  #print(folder)
  index = ch.index(folder)
  labels.append(index)
  skf = StratifiedKFold(n_splits=4, random_state=123, shuffle=True)
balanced_split = skf.split(files, labels)
for ind_train, ind_test in  balanced_split:
    break

In [ ]:
import glob
class DrovaDataset(Dataset):
    def __init__(self, root, transforms, files, labels, indexes):
        super(DrovaDataset, self).__init__()
        self.root = root 
        self.image_names = np.array(files)[indexes]
        self.transforms = transforms
        self.labels = np.array(labels)[indexes]

  
    def __getitem__(self, idx):
        
        sample = {}
        #print(self.image_names[idx])
        image = cv2.imread((os.path.join(self.root, self.image_names[idx])))
        #print(self.image_names[idx])
        # #print(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        sample["image"] = image
        #sample['label'] = self.labels_dict[self.image_names[idx]]
        sample['label'] = self.labels[idx]
        sample['id'] = idx
        if self.transforms is not None:
             sample = self.transforms(sample)

        #print(sample)
       
        return sample

    def __len__(self):
        return len(self.image_names)

In [ ]:
class ScaleMinSideToSize(object):
    def __init__(self, size=(CROP_SIZE, CROP_SIZE), elem_name='image'):
        # self.size = torch.tensor(size, dtype=torch.float)
        self.size = np.asarray(size, dtype=np.float)
        self.elem_name = elem_name

    def __call__(self, sample):
        h, w, _ = sample[self.elem_name].shape
        if h > w:
            f = self.size[0] / w
        else:
            f = self.size[1] / h

        sample[self.elem_name] = cv2.resize(sample[self.elem_name], None, fx=f, fy=f, interpolation=cv2.INTER_AREA)
        sample["scale_coef"] = f

       

        return sample


class CropCenter(object):
    def __init__(self, size=128, elem_name='image'):
        self.size = size
        self.elem_name = elem_name

    def __call__(self, sample):
        img = sample[self.elem_name]
        h, w, _ = img.shape
        margin_h = (h - self.size) // 2
        margin_w = (w - self.size) // 2
        sample[self.elem_name] = img[margin_h:margin_h + self.size, margin_w:margin_w + self.size]
       # sample["crop_margin_x"] = margin_w
       # sample["crop_margin_y"] = margin_h

    
        return sample
        
class TransformByKeys(object):
    def __init__(self, transform, names):
        self.transform = transform
        self.names = set(names)

    def __call__(self, sample):
        for name in self.names:
            if name in sample:
                sample[name] = self.transform(sample[name])

        return sample

In [ ]:
from torchvision import transforms
import albumentations


class apply_albumentations(object):
    def __init__(self, augmenter):
        self.augmenter = augmenter


    def __call__(self, sample):
        #print(sample)
        #print(self.augmenter)
        sample['image'] = self.augmenter(image=np.array(sample['image']))["image"]
        #plt.imshow(sample['image'])
        return sample

augmenter = albumentations.Compose([
    albumentations.ShiftScaleRotate(rotate_limit=25, p=0.7),
    albumentations.RandomBrightnessContrast(p=0.4),
    albumentations.RandomGamma(p=0.4),
    albumentations.Blur(blur_limit=2, p=0.1),
    albumentations.GaussNoise((10, 100), p=0.2),
    albumentations.HorizontalFlip(p=0.5)
])


train_transforms = transforms.Compose([
        apply_albumentations(augmenter),                                
        ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
        CropCenter(CROP_SIZE),
        TransformByKeys(transforms.ToPILImage(), ("image",)),
        TransformByKeys(transforms.ToTensor(), ("image",)),
        #TransformByKeys(transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25]), ("image",)),#
        TransformByKeys(transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ("image",))
])

val_transforms = transforms.Compose([
                                
        ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
        CropCenter(CROP_SIZE),
        TransformByKeys(transforms.ToPILImage(), ("image",)),
        TransformByKeys(transforms.ToTensor(), ("image",)),
        #TransformByKeys(transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25]), ("image",)),#
        TransformByKeys(transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ("image",))
])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


In [ ]:
def train(model, loader, loss_fn, optimizer, device,scheduler):
    model.train()
    train_loss = []
    print_loss = [] 
    #lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)#, mode='min', factor=0.1, patience=10, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    for i, batch in enumerate(tqdm(loader, total=len(loader), desc="training...", position=0 , leave = True)):
        #print(batch)
        images = batch["image"].to(device) 
        labels = batch["label"]  
        


        pred_labels = model(images).cpu()  # B x (2 * NUM_PTS)
        labels_new = torch.zeros_like(pred_labels)
        #print(labels.shape)
        labels_new[range(64), labels] = 1
        #print(labels_new)
        loss1 = loss_fn(pred_labels, labels_new, reduction = 'mean')
        loss = ce(pred_labels, labels)
        train_loss.append(loss.item())

        optimizer.zero_grad()
        loss1.backward()
        optimizer.step()

        print_loss.append(loss.item())
        if (i + 1) % 20 == 0:
            mean_loss = np.mean(print_loss)
            #print(f'Loss: {mean_loss:.7f}')
            scheduler.step(mean_loss)
            print_loss = [] 
        
        
    return np.mean(train_loss)


def validate(model, loader, loss_fn, device):
   model.eval()
   val_mse_loss = []
   val_loss = []
   real = []
   pred = []
   for batch in tqdm(loader, total=len(loader), desc="validation..."):
       #print(batch["image"].shape)
       images = batch["image"].to(device)
       labels = batch["label"]

       with torch.no_grad():
           pred_labels = model(images).cpu()
           loss = ce(pred_labels, labels)
           #loss = loss_fn(pred_labels, labels)

       labels_pred = pred_labels.argmax(1)
       real.extend(labels)
       pred.extend(labels_pred)     
       val_loss.append(loss.item())
   return (np.mean(val_loss))   , (accuracy_score(real, pred))



In [ ]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
def calculate_predictions(model, loader):
    model.eval()
    epoch_loss = 0
    history = []
    real = []
    pred = []
    ids = []
    scores = []
    with torch.no_grad():

        for batch in tqdm(loader, total=len(loader), desc="validation..."):
             images = batch["image"].to(device)
             labels = batch["label"]
             idx = batch["id"]

         

             pred_labels = model(images).cpu()
             labels_pred = pred_labels.argmax(1)

            
             real.extend(labels)
             pred.extend(labels_pred) 
             ids.extend(idx)
             #print(pred_labels.shape)
             #print(pred_labels[range(32), labels].shape)
             
             if labels.shape[0] == args.batch_size:
                   scores.extend(list(pred_labels[range(args.batch_size), labels].numpy()))
             
        #print(accuracy_score(real, pred)) 
        print(confusion_matrix(real, pred))  
        print(classification_report(real, pred))  
        #f1 = ((f1_score(real, pred, average = 'binary', pos_label = 0))  + (f1_score(real, pred, average = 'binary', pos_label = 1)))/2
        return (accuracy_score(real, pred)), ids, real, pred , scores
        #plt.hist(real)

#Обучение, валидация

In [ ]:
import torchvision

name_model = "drova_focal_3"

train_dataset = DrovaDataset('./data/', train_transforms, files, labels, ind_train)
val_dataset = DrovaDataset('./data/', val_transforms, files, labels, ind_test)

train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                              pin_memory=True, shuffle=True, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                              pin_memory=True, shuffle=False, drop_last=False)
model = models.resnet18(pretrained=True)
#model = models.squeezenet1_0(pretrained=True)
model.requires_grad_(True)

model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES, bias=True)
model.fc.requires_grad_(True)

for param in model.parameters():
  param.requires_grad = True

for param in model.fc.parameters():
  param.requires_grad = True

model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.0001, amsgrad=True)
ce =  nn.CrossEntropyLoss(reduce = 'mean')
loss_fn = torchvision.ops.sigmoid_focal_loss


#weight = torch.tensor([69/578, 299/578, 210/578], dtype=torch.float, device='cpu')
#ce = nn.CrossEntropyLoss(weight=weight)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=20, factor=0.5, verbose=True)

args.epochs = 10
# 2. train & validate
print("FIRST TRAINING TO FIND BAD LABELS")
best_val_loss = -10000
for epoch in range(args.epochs):
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device, scheduler)
    val_loss, acc = validate(model, val_dataloader, loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.2}\tval loss: {:5.2}  acc = {:5.2}".format(epoch, train_loss, val_loss, acc))
    if acc > best_val_loss:
        best_val_loss = acc
        with open(os.path.join("runs", name_model), "wb") as fp:
            torch.save(model.state_dict(), fp)




Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

FIRST TRAINING TO FIND BAD LABELS


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


training...:   0%|          | 0/6 [00:00<?, ?it/s]

...

validation...:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch # 8:	train loss: 0.013	val loss: 0.096  acc =  0.96


training...:   0%|          | 0/6 [00:00<?, ?it/s]

validation...:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch # 9:	train loss:  0.02	val loss: 0.099  acc =  0.96


In [ ]:
with open(os.path.join("runs", "drova_focal_3"), "rb") as fp:
      best_state_dict = torch.load(fp, map_location="cpu")
      model.load_state_dict(best_state_dict)

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.0001, amsgrad=True)
#best_val_loss = -np.inf
for epoch in range(3):
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device, scheduler)
    val_loss, acc = validate(model, test_dataloader, loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.2}\tval loss: {:5.2}acc =,  {:5.2}".format(epoch, train_loss, val_loss, acc))
    if acc > best_val_loss:
        best_val_loss = acc
        with open(os.path.join("runs", name_model), "wb") as fp:
            torch.save(model.state_dict(), fp)

FIRST TRAINING TO FIND BAD LABELS


training...:   0%|          | 0/6 [00:00<?, ?it/s]

validation...:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch # 0:	train loss:  0.14	val loss:  0.17acc =,   0.94


training...:   0%|          | 0/6 [00:00<?, ?it/s]

validation...:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch # 1:	train loss: 0.081	val loss:  0.12acc =,   0.96


training...:   0%|          | 0/6 [00:00<?, ?it/s]

validation...:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch # 2:	train loss: 0.051	val loss:  0.11acc =,   0.98


In [ ]:
 with open(os.path.join("runs", "drova_resnet18_98_0.25_0.11_focal"), "wb") as fp:
            torch.save(model.state_dict(), fp)
            

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.0001, amsgrad=True)
#best_val_loss = -np.inf
for epoch in range(3):
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device, scheduler)
    val_loss, acc = validate(model, test_dataloader, loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.2}\tval loss: {:5.2}acc =,  {:5.2}".format(epoch, train_loss, val_loss, acc))
    if acc > best_val_loss:
        best_val_loss = acc
        with open(os.path.join("runs", name_model), "wb") as fp:
            torch.save(model.state_dict(), fp)

FIRST TRAINING TO FIND BAD LABELS


training...:   0%|          | 0/6 [00:00<?, ?it/s]

validation...:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch # 0:	train loss: 0.049	val loss:   0.1acc =,   0.97


training...:   0%|          | 0/6 [00:00<?, ?it/s]

validation...:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch # 1:	train loss: 0.047	val loss: 0.097acc =,   0.97


training...:   0%|          | 0/6 [00:00<?, ?it/s]

validation...:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch # 2:	train loss: 0.037	val loss: 0.091acc =,   0.97


In [ ]:
#optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.0001, amsgrad=True)
#best_val_loss = -np.inf
for epoch in range(3):
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device, scheduler)
    val_loss, acc = validate(model, test_dataloader, loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.2}\tval loss: {:5.2}acc =,  {:5.2}".format(epoch, train_loss, val_loss, acc))
    if acc > best_val_loss:
        best_val_loss = acc
        with open(os.path.join("runs", name_model), "wb") as fp:
            torch.save(model.state_dict(), fp)

FIRST TRAINING TO FIND BAD LABELS


training...:   0%|          | 0/6 [00:00<?, ?it/s]

validation...:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch # 0:	train loss: 0.047	val loss: 0.089acc =,   0.97


training...:   0%|          | 0/6 [00:00<?, ?it/s]

validation...:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch # 1:	train loss: 0.045	val loss: 0.088acc =,   0.97


training...:   0%|          | 0/6 [00:00<?, ?it/s]

validation...:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch # 2:	train loss: 0.049	val loss:  0.09acc =,   0.97


Достаем сохраненную модель с наивысшим accuracy 0.98

In [ ]:
with open(os.path.join("runs", "drova_resnet18_98_0.25_0.11_focal"), "rb") as fp:
      best_state_dict = torch.load(fp, map_location="cpu")
      model.load_state_dict(best_state_dict)

In [ ]:
acc, _, _, _,_ = calculate_predictions(model, val_dataloader)
print(acc)

validation...:   0%|          | 0/3 [00:00<?, ?it/s]

[[75  0  0]
 [ 1 51  0]
 [ 1  0 17]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.99        75
           1       1.00      0.98      0.99        52
           2       1.00      0.94      0.97        18

    accuracy                           0.99       145
   macro avg       0.99      0.98      0.98       145
weighted avg       0.99      0.99      0.99       145

0.9862068965517241


#Получение submission.csv на тесте

In [ ]:
files_test = []
root_test = '/content/data/test/test/'

In [ ]:
files_test = [f'/content/data/test/test/{i}.png' for i in range(1, 250)] # glob.glob(os.path.join(root_test, f"*.png"))

In [ ]:
print(files_test)

['/content/data/test/test/1.png', '/content/data/test/test/2.png', '/content/data/test/test/3.png', '/content/data/test/test/4.png', '/content/data/test/test/5.png', '/content/data/test/test/6.png', '/content/data/test/test/7.png', '/content/data/test/test/8.png', '/content/data/test/test/9.png', '/content/data/test/test/10.png', '/content/data/test/test/11.png', '/content/data/test/test/12.png', '/content/data/test/test/13.png', '/content/data/test/test/14.png', '/content/data/test/test/15.png', '/content/data/test/test/16.png', '/content/data/test/test/17.png', '/content/data/test/test/18.png', '/content/data/test/test/19.png', '/content/data/test/test/20.png', '/content/data/test/test/21.png', '/content/data/test/test/22.png', '/content/data/test/test/23.png', '/content/data/test/test/24.png', '/content/data/test/test/25.png', '/content/data/test/test/26.png', '/content/data/test/test/27.png', '/content/data/test/test/28.png', '/content/data/test/test/29.png', '/content/data/test/te

In [ ]:
files_test[119] ='/content/data/test/test/120.JPG'

In [ ]:
labels_test = pd.read_csv("submission_combined.csv")['class'].values
#print(labels_test)
ch2 = [1, 3, 0]
labels_test_new = [ch2.index(label) for label in labels_test]

In [ ]:
print(labels_test_new)

[2, 1, 2, 1, 0, 0, 1, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 1, 2, 2, 2, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 2, 1, 0, 1, 0, 0, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 0, 2, 0, 1, 1, 0, 1, 0, 1, 0, 2, 1, 0, 1, 0, 1, 0, 0, 0, 1, 2, 1, 1, 1, 0, 0, 0, 1, 1, 0, 2, 0, 0, 0, 0, 2, 1, 1, 0, 0, 1, 1, 1, 2, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 0, 2, 1, 1, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 2, 2, 2, 0, 2, 2, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 2, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 2, 1, 0, 1, 0, 0, 1, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0]


In [ ]:
#
ind_test1 = range(249)
test_dataset = DrovaDataset('./data/test/test/', val_transforms, files_test, labels_test_new, ind_test1)
test_dataloader = DataLoader(test_dataset, batch_size=args.batch_size, num_workers=1,
                              pin_memory=True, shuffle=False, drop_last=False)


In [ ]:
acc, _, _, pred,_ = calculate_predictions(model, test_dataloader)

validation...:   0%|          | 0/4 [00:00<?, ?it/s]

[[129   0   3]
 [  2  87   0]
 [  1   0  27]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       132
           1       1.00      0.98      0.99        89
           2       0.90      0.96      0.93        28

    accuracy                           0.98       249
   macro avg       0.96      0.97      0.97       249
weighted avg       0.98      0.98      0.98       249



In [ ]:
ch = ['1', '3', 'd']
ch1 = [1, 3, 0]

In [ ]:
pred_indexes = [ch1[int(pred[i])] for i in range(249)] 

In [ ]:
import pandas as pd
result = pd.DataFrame(columns = ['id','class'])
result['id']= range(1, 250)
result['class'] = pred_indexes


In [ ]:
result.to_csv("test_resnet18_98_0.25_0.11_focal.csv", index = None)

#Результаты

В результате многочисленных запусков различных конфигураций resnet18 с изменениями расписания шага и гиперпараметров, получились сабмиты, которые дали на паблике следующие лучшие скоры:   
```
0.99328, 0.98657.
```

Решено было объединить данные сабмиты в  ансамбль методом голосования  с сабмитами на основе resnet50, vgg16, resnet18. 
В результате нами было получено два сабмита  со скорами на паблике:  

```
*   public 1.00000, private 0.97000
*   public 0.99328  private 0.98000
```



